# Introdução:

Retomar arquivo anterior, com link, explicar talvez o funcionamento simples de uma rede neural, uso do Pytorch pra otimizar cálculos, Optuna pra otimizar hiperparâmetros, métrica de erro

# Desenvolvimento:

## Importações e definições:

Importando as bibliotecas necessárias:

In [3]:
import pandas as pd
import numpy as np
import os
import optuna

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
import torch.optim as optim


Definindo as constantes do problema:

In [4]:
TAMANHO_TESTE = 0.1
TAMANHO_VALIDACAO = 0.1

SEMENTE_ALEATORIA = 1249

NUM_EPOCAS = 50

## Pré-processamento:

Carregando o dataset de energia:

In [9]:
TARGET = ["Storage_Efficiency_Percentage"]
FEATURES = [
    "Type_of_Renewable_Energy",
    "Installed_Capacity_MW",
    # Ver com Cassar: Dados sequenciais
    #"Energy_Production_MWh",
    #"Energy_Consumption_MWh",
    #"Energy_Storage_Capacity_MWh",
    "Grid_Integration_Level",
    "Initial_Investment_USD",
    # Ver se vale a pena
    #"Funding_Sources",
    "Financial_Incentives_USD",
    "GHG_Emission_Reduction_tCO2e",
    "Air_Pollution_Reduction_Index",
    #"Jobs_Created",
]

df_energia = pd.read_csv('energy_dataset_.csv')
df_energia = df_energia.reindex(FEATURES + TARGET, axis=1)
df_energia = df_energia.dropna()
df_energia

,Type_of_Renewable_Energy,Installed_Capacity_MW,Grid_Integration_Level,Initial_Investment_USD,Financial_Incentives_USD,GHG_Emission_Reduction_tCO2e,Air_Pollution_Reduction_Index,Storage_Efficiency_Percentage
0,4,93.423205,4,4.732248e+08,9.207772e+06,6663.816572,81.742461,89.887562
1,4,590.468942,4,1.670697e+08,1.685101e+06,30656.049820,78.139042,84.403343
2,1,625.951142,2,8.463610e+07,5.111813e+06,1749.613759,8.461296,60.498249
3,1,779.998728,3,3.967690e+08,4.805902e+06,43233.237820,8.402441,86.897861
4,3,242.106837,2,3.574413e+07,1.668601e+07,14858.662760,28.822867,70.949351
...,...,...,...,...,...,...,...,...
14995,3,745.032555,4,3.484136e+08,1.558508e+07,25234.911810,78.923200,90.791405
14996,1,15.187023,4,2.560179e+08,6.866618e+06,15762.519790,54.982974,78.252040
14997,3,877.539059,1,1.300112e+08,3.837764e+06,44597.809410,43.915897,58.282928
14998,7,551.264716,2,3.334831e+08,5.347706e+06,34363.858000,4.877145,73.573666


Aplicando a normalização por Máximo Absoluto considerando as colunas fixas, que contém dados discretos e necessários para o treinamento e interpretabilidade do algoritmo:

In [ ]:
df_energia_normalizado = df_energia.copy()
colunas_fixas = ['Type_of_Renewable_Energy', 'Grid_Integration_Level','Funding_Sources']
colunas_variaveis = df_energia_normalizado.columns.difference(colunas_fixas)

normalizador = StandardScaler()
df_energia_normalizado[colunas_variaveis] = normalizador.fit_transform(df_energia_normalizado[colunas_variaveis])
df_energia_normalizado

In [14]:
# df_energia_normalizado = df_energia.copy()
# colunas_fixas = ['Type_of_Renewable_Energy', 'Grid_Integration_Level','Funding_Sources']
# colunas_variaveis = df_energia_normalizado.columns.difference(colunas_fixas)

# normalizador = MaxAbsScaler()
# df_energia_normalizado[colunas_variaveis] = normalizador.fit_transform(df_energia_normalizado[colunas_variaveis])
# df_energia_normalizado

Dividindo os dados em treino, validação e teste, permitindo que a rede neural se ajuste aos dados de treino e seja otimizada para os dados de validação, evitando o *overfitting*:

In [13]:
# Separação dos dados de teste
indices = df_energia.index
indices_treino_val, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino_val = df_energia.loc[indices_treino_val]
df_teste = df_energia.loc[indices_teste]

X_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values

In [15]:
# Separação dos dados de treino e validação
indices = df_treino_val.index
indices_treino, indices_val = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df_energia.loc[indices_treino]
df_val = df_energia.loc[indices_val]

X_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values

X_val = df_val.reindex(FEATURES, axis=1).values
y_val = df_val.reindex(TARGET, axis=1).values

Fazendo uma normalização por Máximo Absoluto, ajustando aos dados de treino e transformando aos demais dados:

In [17]:
norm_x = MaxAbsScaler()
norm_x.fit(X_treino)

norm_y = MaxAbsScaler()
norm_y.fit(y_treino)

X_treino = norm_x.transform(X_treino)
y_treino = norm_y.transform(y_treino)

X_val = norm_x.transform(X_val)
y_val = norm_y.transform(y_val)

X_teste = norm_x.transform(X_teste)
y_teste = norm_y.transform(y_teste)


Convertendo os dados para Tensores, uma estrutura especial utilizada no módulo `Pytorch`:

In [18]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)

X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

## Optuna:

In [ ]:
# Função que o Optuna vai otimizar
def objective(trial):
    # Hiperparâmetros a serem testados
    n_camadas = trial.suggest_int("n_layers", 1, 4)
    tamanho_camada_oculta = trial.suggest_int("hidden_size", 16, 128)
    funcao_ativacao = trial.suggest_categorical("activation", ["ReLU", "Tanh"])
    taxa_aprendizado = trial.suggest_float("lr", 1e-4, 1e-2)

    # Montando a rede com base nos hiperparâmetros
    layers = []
    input_size = X.shape[1]
    for i in range(n_camadas):
        layers.append(nn.Linear(input_size if i == 0 else tamanho_camada_oculta, tamanho_camada_oculta))
        layers.append(getattr(nn, funcao_ativacao)())
    layers.append(nn.Linear(tamanho_camada_oculta, 1))
    model = nn.Sequential(*layers)

    # Treinamento
    otimizador = optim.Adam(model.parameters(), lr=taxa_aprendizado)
    fun_perda = nn.MSELoss()
    
    for epoch in range(50):  
        model.train()
        y_pred = model(torch.tensor(X_treino, dtype=torch.float32))
        loss = fun_perda(y_pred, torch.tensor(y_teste, dtype=torch.float32))
        otimizador.zero_grad()
        loss.backward()
        otimizador.step()
    
    return loss.item()

In [ ]:
study = optuna.create_study(direction="minimize")

[I 2025-05-20 09:43:38,374] A new study created in memory with name: no-name-c2a66739-7af0-4d0b-bd28-22ef187a90fd


In [ ]:
# SO RODAR QUANDO FOR NECESSÁRIO

#study.optimize(objective, n_trials=100)  # Faz 100 testes de arquitetura

#print("Melhores hiperparâmetros:", study.best_params)

## Rede Neural MLP:

## Treinamento (Talvez):

## Curva de Aprendizado (Talvez):

## Testando a Rede:

# Conclusões:

# Referências:

[1] CASSAR, DANIEL. "". Material de aula, Ano

[2] 

[3] 